# 테스트

In [7]:
from bs4 import BeautifulSoup

html = """
<html>
  <body>
    <div id="main" class="content">
      <h1>첫 번째 헤더</h1>
      <h1 class="header">두 번째 헤더</h1>
      <p class="description">설명입니다.</p>
      <a href="https://example1.com" class="link">링크 1</a>
      <a href="https://example2.com" class="link">링크 2</a>
      <a href="https://example3.com" class="external">링크 3</a>
    </div>
    <div id="main2" class="content">
      <div class="items">
        <div>
            <img class="thumbnail" src="/data/picture1.png" />
            <div class="title">타이틀1</div>
            <div class="description">설명1</div>
        </div>
        <div>
            <img class="thumbnail" src="/data/picture2.png" />
            <div class="title">타이틀2</div>
            <div class="description">설명2</div>
        </div>
        <div>
            <img class="thumbnail" src="/data/picture3.png" />
            <div class="title test">타이틀3</div>
            <div class="description">설명3</div>
        </div>
      </div>
    </div>
  </body>
</html>
"""

# 특정하지 않으면 처음 한개만 추출된
soup = BeautifulSoup(html, 'html.parser')
soup.find('div', class_='title')

<div class="title">타이틀1</div>

In [ ]:
#첫번째 img 추출(둘 다 같음)
soup.find('img')
soup.find('img', src='/data/picture1.png')

<img class="thumbnail" src="/data/picture1.png"/>

In [10]:
# class가 title인 div를 모두 추출해주세요
soup.find_all("div", class_="title")

[<div class="title">타이틀1</div>,
 <div class="title">타이틀2</div>,
 <div class="title test">타이틀3</div>]

In [11]:
# class가 items인 div 안의 div들을 모두 추출해주세요
# class가 items > div를 가져온다.
items = soup.find("div", class_="items")
div_list = items.find_all("div")
for div in div_list:
    print(div)
    print("---")

<div>
<img class="thumbnail" src="/data/picture1.png"/>
<div class="title">타이틀1</div>
<div class="description">설명1</div>
</div>
---
<div class="title">타이틀1</div>
---
<div class="description">설명1</div>
---
<div>
<img class="thumbnail" src="/data/picture2.png"/>
<div class="title">타이틀2</div>
<div class="description">설명2</div>
</div>
---
<div class="title">타이틀2</div>
---
<div class="description">설명2</div>
---
<div>
<img class="thumbnail" src="/data/picture3.png"/>
<div class="title test">타이틀3</div>
<div class="description">설명3</div>
</div>
---
<div class="title test">타이틀3</div>
---
<div class="description">설명3</div>
---


In [ ]:
# img 태그에서 src를 추출
print(soup.find("img").attrs) # dict 형태로 추출
print(soup.find("img").attrs["src"])
print(soup.find("img").get("src")) # 에러가 발생하지 않음 => None 응답
print(soup.find("img")["src"])

{'class': ['thumbnail'], 'src': '/data/picture1.png'}


KeyError: 'test'